In [1]:
import numpy as np
import ipywidgets.widgets as widgets
import time
from IPython.display import display
image_display = widgets.Image(format="jpeg", width="45%")
full_display = widgets.Image(format="jpeg", width="45%")
display(widgets.HBox([image_display, full_display]))

location_coords_display = widgets.Label()
display(location_coords_display)

import cv2
import pyzed.sl as sl
camera = sl.Camera()
camera_params = sl.InitParameters()
camera_params.camera_resolution = sl.RESOLUTION.VGA
camera_params.depth_mode = sl.DEPTH_MODE.ULTRA
camera_params.coordinate_units = sl.UNIT.MILLIMETER

camera_status = camera.open(camera_params)
if camera_status != sl.ERROR_CODE.SUCCESS:
    print("camera error")
    print(camera_status)
    camera.close()
    exit()

# initialize robot
import motors
robot = motors.MotorsYukon()
robot.stop()

# initialize model
import tracking
m = tracking.Model()

# get initial image and choose object to track
image_mat = sl.Mat()
depth_mat = sl.Mat()
started_tracking = False
while not started_tracking:
    err = camera.grab()
    if err == sl.ERROR_CODE.SUCCESS:
        camera.retrieve_image(image_mat)
        image = image_mat.get_data()
        
        image_display.value = m.show_all_boxes(image)
    
        user_input = input("enter id to track (or leave blank to skip):")
        if user_input == "":
            continue
        else:
            m.tracked_id = int(user_input)
            print("tracking object id " + str(user_input))
            started_tracking = True

# movement parameters
KEEP_X_BETWEEN = (275, 417)
TURN_SPEED = 0.3
TURN_SLEEP = 0.05
KEEP_DEPTH_BETWEEN = (750, 1250)
MOVE_SPEED = 0.5
MOVE_SLEEP = 0.05

# start tracking
running = True
while running:
    err = camera.grab()
    if err == sl.ERROR_CODE.SUCCESS:
        camera.retrieve_image(image_mat)
        image = image_mat.get_data()

        camera.retrieve_measure(depth_mat, measure=sl.MEASURE.DEPTH)
        depth = depth_mat.get_data()
        
        tracked_box = m.track(image, return_type="corners")
        
        if tracked_box is False:
            image_display.value = bytes(cv2.imencode('.jpg', image)[1])
            robot.stop()
        else:
            x_centre = int((tracked_box[0] + tracked_box[2]) / 2)
            y_centre = int((tracked_box[1] + tracked_box[3]) / 2)
            image_rect = cv2.rectangle(
                image,
                (int(tracked_box[0]), int(tracked_box[1])),
                (int(tracked_box[2]), int(tracked_box[3])),
                (255, 0, 0),
                4
            )
            image_display.value = bytes(cv2.imencode('.jpg', image_rect)[1])

            # turn if tracked person is outside KEEP_X_BETWEEN
            if x_centre < KEEP_X_BETWEEN[0]:
                robot.left(speed=TURN_SPEED)
                time.sleep(TURN_SLEEP)
                print("turning left   ", end="\r")
            elif x_centre > KEEP_X_BETWEEN[1]:
                robot.right(speed=TURN_SPEED)
                time.sleep(TURN_SLEEP)
                print("turning right  ", end="\r")
            else:
                # only correct distance if target is centred
                depth_value = depth[y_centre, x_centre]
                print(depth_value)
                if depth_value < KEEP_DEPTH_BETWEEN[0]:
                    robot.backward(speed=MOVE_SPEED)
                    time.sleep(MOVE_SLEEP)
                    print("turning back   ", end="\r")
                elif depth_value > KEEP_DEPTH_BETWEEN[1]:
                    robot.forward(speed=MOVE_SPEED)
                    time.sleep(MOVE_SLEEP)
                    print("turning forward", end="\r")
                else:
                    robot.stop()
                    print("stopping       ", end="\r")

        full_display.value = m.show_all_boxes(image)
    

camera.close()

Label(value='')

[2025-05-07 08:37:32 UTC][ZED][INFO] Logging level INFO
[2025-05-07 08:37:32 UTC][ZED][INFO] Logging level INFO
[2025-05-07 08:37:32 UTC][ZED][INFO] Logging level INFO
[2025-05-07 08:37:33 UTC][ZED][INFO] [Init]  Depth mode: ULTRA
[2025-05-07 08:37:34 UTC][ZED][INFO] [Init]  Camera successfully opened.
[2025-05-07 08:37:34 UTC][ZED][INFO] [Init]  Camera FW version: 1523
[2025-05-07 08:37:34 UTC][ZED][INFO] [Init]  Video mode: VGA@100
[2025-05-07 08:37:34 UTC][ZED][INFO] [Init]  Serial Number: S/N 38559894
WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading yolo11n.engine for TensorRT inference...
[05/07/2025-09:37:39] [TRT] [I] Loaded engine size: 12 MiB
[05/07/2025-09:37:39] [TRT] [W] Using an engine plan file across different models of devices is not recommended and is likely to affect performance or even cause errors.
[05/07/2025-09:37:39] [TRT] [I] [Mem

enter id to track (or leave blank to skip): 2


tracking object id 2
1459.4795
1414.777forward
1118.3726orward
1037.7329      
1049.0729      
1078.8495      
1110.1952      
nanpping       
nanpping       
1036.1146eft   
nanpping       
1005.456       
848.2844       
908.19446      
881.5559       
875.5593       
881.42163      
878.6243       
882.22766      
877.4361       
nanpping       
974.0058left   
nanpping       
nanpping       
1825.108       
956.7066forward
976.5276left   
926.931g       
871.3802right  
868.02997      
1464.9944      
891.28204ight  
nanpping       
4622.6265      
774.1025forward
737.8226right  
525.57745ack   
nanning back   
nanpping       
628.2897       
726.56836ack   
nanning back   
nanpping       
1342.7598      
nanning forward
1259.9983      
1251.442forward
1122.1848orward
2091.4072      
842.8548left   
727.8277       
nanning back   
nanpping       
nanpping       
831.7463left   
1294.4166ight  
new box id: tensor(36.)
    coords:  tensor([187.8112,   9.7858, 311.0752, 308.8964])
las

KeyboardInterrupt: 